# IPython Widget Events

## 特殊事件

`Button` 不是用来表示数据类型的。相反，按钮小部件用于处理鼠标点击。`Button` 的 `on_click` 方法可以用来注册当按钮被点击时要调用的函数。`on_click` 的文档字符串可以在下面看到。

In [1]:
# Imports for JupyterLite
%pip install -q ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [2]:
import ipywidgets as widgets
print(widgets.Button.on_click.__doc__)

Note: you may need to restart the kernel to use updated packages.


### 示例

由于按钮点击是无状态的，它们通过自定义消息从前端传输到后端。通过使用 `on_click` 方法，下面展示了在点击时打印消息的按钮。要捕获 `print` （或其他任何类型的输出）并确保其显示，请确保将其发送到 `Output` 小部件（或将您想要显示的信息放入 `HTML` 小部件）。

In [3]:
from IPython.display import display
import ipywidgets as widgets
button = widgets.Button(description="Click Me!")
output = widgets.Output()

display(button, output)

def on_button_clicked(b):
    with output:
        print("Button clicked.")

button.on_click(on_button_clicked)

Button(description='Click Me!', style=ButtonStyle())

Output()

## Traitlet 事件

小部件属性是 IPython traitlets，而 traitlets 是有事件的。要处理变化，可以使用小部件的 `observe` 方法注册回调函数。`observe` 的文档字符串可以在下面看到。

In [4]:
print(widgets.Widget.observe.__doc__)

Setup a handler to be called when a trait changes.

        This is used to setup dynamic notifications of trait changes.

        Parameters
        ----------
        handler : callable
            A callable that is called when a trait changes. Its
            signature should be ``handler(change)``, where ``change`` is a
            dictionary. The change dictionary at least holds a 'type' key.
            * ``type``: the type of notification.
            Other keys may be passed depending on the value of 'type'. In the
            case where type is 'change', we also have the following keys:
            * ``owner`` : the HasTraits instance
            * ``old`` : the old value of the modified trait attribute
            * ``new`` : the new value of the modified trait attribute
            * ``name`` : the name of the modified trait attribute.
        names : list, str, All
            If names is All, the handler will apply to all traits.  If a list
            of str, handler wil

### 签名

在文档字符串中提到，注册的回调函数必须具有 `handler(change)` 的形式，其中 `change` 是一个字典，用于保存有关更改的信息。

通过使用该方法，下面是一个示例，展示了如何在 `IntSlider` 的值发生变化时输出其值。

In [5]:
int_range = widgets.IntSlider()
output2 = widgets.Output()

display(int_range, output2)

def on_value_change(change):
    with output2:
        print(change['new'])

int_range.observe(on_value_change, names='value')

IntSlider(value=0)

Output()

## 链接小部件

通常，您可能希望简单地将小部件属性链接在一起。与使用裸 `traitlets` 事件相比，属性同步可以通过更简单的方式进行。

### 在内核中链接 `traitlets` 属性

第一种方法是使用 `traitlets` 模块中的 `link` 和 `dlink` 函数（这两个函数由 `ipywidgets` 模块重新导出以方便使用）。这只适用于我们正在与实时内核进行交互的情况。

In [6]:
caption = widgets.Label(value='The values of slider1 and slider2 are synchronized')
sliders1, slider2 = widgets.IntSlider(description='Slider 1'),\
                    widgets.IntSlider(description='Slider 2')
l = widgets.link((sliders1, 'value'), (slider2, 'value'))
display(caption, sliders1, slider2)

Label(value='The values of slider1 and slider2 are synchronized')

IntSlider(value=0, description='Slider 1')

IntSlider(value=0, description='Slider 2')

In [7]:
caption = widgets.Label(value='Changes in source values are reflected in target1')
source, target1 = widgets.IntSlider(description='Source'),\
                  widgets.IntSlider(description='Target 1')
dl = widgets.dlink((source, 'value'), (target1, 'value'))
display(caption, source, target1)

Label(value='Changes in source values are reflected in target1')

IntSlider(value=0, description='Source')

IntSlider(value=0, description='Target 1')

`traitlets.link` 和 `traitlets.dlink` 函数返回一个 `Link` 或 `DLink` 对象。通过调用 `unlink` 方法可以断开链接。

In [8]:
l.unlink()
dl.unlink()

### 在内核中注册回调以响应 `trait` 的变化

由于 Python 端的部件属性是 `traitlets`，因此每当模型从前端获取更新时，您可以注册处理程序来处理更改事件。

传递给 `observe` 的处理程序将使用 `change` 参数进行调用。`change` 对象至少包含 `type` 键和 `name` 键，分别对应于通知的类型和触发通知的属性的名称。

根据 `type` 的值，可能会传递其他键。在 `type` 为 `change` 的情况下，我们还有以下几个键：

- `owner` : `HasTraits` 实例
- `old` : 修改后的 `trait` 属性的旧值
- `new` : 修改后的 `trait` 属性的新值
- `name` : 修改后的 `trait` 属性的名称。

In [9]:
caption = widgets.Label(value='The slider value is in its initial position.')
slider = widgets.IntSlider(min=-5, max=5, value=1, description='Slider')

def handle_slider_change(change):
    caption.value = 'The slider value is ' + (
        'negative' if change.new < 0 else 'nonnegative'
    )

slider.observe(handle_slider_change, names='value')

display(caption, slider)

Label(value='The slider value is in its initial position.')

IntSlider(value=1, description='Slider', max=5, min=-5)

### 从客户端链接小部件属性

在同步 `traitlets` 属性时，由于往返服务器端的延迟，您可能会遇到延迟。您还可以使用链接小部件直接在浏览器中链接小部件属性，可以选择单向或双向方式。

在没有内核的情况下将小部件嵌入到 `html` 网页中时，Javascript 链接会持续存在。

In [10]:
caption = widgets.Label(value='The values of range1 and range2 are synchronized')
range1, range2 = widgets.IntSlider(description='Range 1'),\
                 widgets.IntSlider(description='Range 2')
l = widgets.jslink((range1, 'value'), (range2, 'value'))
display(caption, range1, range2)

Label(value='The values of range1 and range2 are synchronized')

IntSlider(value=0, description='Range 1')

IntSlider(value=0, description='Range 2')

In [11]:
caption = widgets.Label(value='Changes in source_range values are reflected in target_range1')
source_range, target_range1 = widgets.IntSlider(description='Source range'),\
                              widgets.IntSlider(description='Target range 1')
dl = widgets.jsdlink((source_range, 'value'), (target_range1, 'value'))
display(caption, source_range, target_range1)

Label(value='The values of range1 and range2 are synchronized')

IntSlider(value=0, description='Range 1')

IntSlider(value=0, description='Range 2')

`widgets.jslink` 函数返回 `Link` 小部件。通过调用 `unlink` 方法可以断开链接。

In [12]:
# l.unlink()
# dl.unlink()

### 内核链接和客户端链接的区别

内核链接是指通过 Python 进行链接。如果在内核中链接了两个滑块，当一个滑块发生变化时，浏览器会向内核（在这种情况下是 Python）发送消息，更新变化的滑块，然后内核中的链接小部件将更改传播到内核中的另一个滑块对象，然后另一个滑块的内核对象会向浏览器发送消息，以更新浏览器中另一个滑块的视图。如果内核没有运行（如在静态网页中），那么控件将不会被链接。

使用 `jslink` （即，在浏览器端）进行链接意味着在 JavaScript 中构建链接。当一个滑块发生变化时，浏览器中运行的 JavaScript 会更改浏览器中另一个滑块的值，完全不需要与内核进行通信。如果滑块附加到内核对象，每个滑块将独立地更新它们在内核端的对象。

要了解两者之间的区别，请访问 `ipywidgets` 文档中此页面的[静态版本](http://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Events.html)，并尝试使用页面底部附近的滑块。使用 `link` 和 `dlink` 在内核中链接的那些滑块不再链接，但是使用 `jslink` 和 `jsdlink` 在浏览器中链接的那些滑块仍然保持链接。

## 持续更新

一些部件提供了在其 `continuous_update` 属性中进行选择的选项，可以在持续更新值和仅在用户提交值时（例如，通过按下 Enter 键或从控件导航离开）才更新值之间进行选择。在接下来的示例中，我们看到“延迟”控件仅在用户完成拖动滑块或提交文本框后传输其值。而“连续”控件则在其值发生变化时持续传输它们的值。尝试在每个文本框中输入一个两位数，或拖动每个滑块，以观察差异。

In [13]:
a = widgets.IntSlider(description="Delayed", continuous_update=False)
b = widgets.IntText(description="Delayed", continuous_update=False)
c = widgets.IntSlider(description="Continuous", continuous_update=True)
d = widgets.IntText(description="Continuous", continuous_update=True)

widgets.link((a, 'value'), (b, 'value'))
widgets.link((a, 'value'), (c, 'value'))
widgets.link((a, 'value'), (d, 'value'))
widgets.VBox([a,b,c,d])

滑块、`Text` 和 `Textarea` 控件默认设置为 `continuous_update=True`。`IntText` 和其他用于输入整数或浮点数的文本框默认设置为 `continuous_update=False`（因为通常你会想在按下 Enter 键或导航离开文本框之前输入完整的数字）。

## 去抖动

当特质变化触发执行繁重计算的回调函数时，你可能不希望像值更新那样频繁地进行计算。例如，如果特质由一个滑块驱动，并且其 `continuous_update` 设置为 `True`，用户将触发一连串快速连续的计算。

去抖动通过延迟回调执行直到值在一定时间内没有改变来解决这个问题，之后使用最新值调用回调函数。效果是只有当特质暂停改变一定时间后才会调用回调函数。

去抖动可以使用异步循环或线程来实现。下面展示了一种更适合 `ipywidgets` 的异步解决方案。如果你希望改用线程来进行去抖动，可以将 `Timer` 类替换为 `from threading import Timer`。

In [14]:
import asyncio

class Timer:
    def __init__(self, timeout, callback):
        self._timeout = timeout
        self._callback = callback

    async def _job(self):
        await asyncio.sleep(self._timeout)
        self._callback()

    def start(self):
        self._task = asyncio.ensure_future(self._job())

    def cancel(self):
        self._task.cancel()

def debounce(wait):
    """ Decorator that will postpone a function's
        execution until after `wait` seconds
        have elapsed since the last time it was invoked. """
    def decorator(fn):
        timer = None
        def debounced(*args, **kwargs):
            nonlocal timer
            def call_it():
                fn(*args, **kwargs)
            if timer is not None:
                timer.cancel()
            timer = Timer(wait, call_it)
            timer.start()
        return debounced
    return decorator

以下是我们如何使用 `debounce` 函数作为装饰器。尝试更改滑块的值。文本框将在滑块暂停大约 0.2 秒后才会更新。

In [15]:
slider = widgets.IntSlider()
text = widgets.IntText()

@debounce(0.2)
def value_changed(change):
    text.value = change.new
slider.observe(value_changed, 'value')

widgets.VBox([slider, text])

## 节流

节流(Throttling)是另一种可用于限制回调的技术。与去抖动不同，去抖动在自上一次（尝试）调用函数以来未经过一定时间时会忽略对函数的调用，而节流则会限制调用的速率。这确保了函数被定期调用。

下面展示了一个同步解决方案。同样地，如果你想使用线程而不是异步编程，你可以用 `from threading import Timer` 替换 `Timer` 类。

In [16]:
import asyncio
from time import time

def throttle(wait):
    """ Decorator that prevents a function from being called
        more than once every wait period. """
    def decorator(fn):
        time_of_last_call = 0
        scheduled, timer = False, None
        new_args, new_kwargs = None, None
        def throttled(*args, **kwargs):
            nonlocal new_args, new_kwargs, time_of_last_call, scheduled, timer
            def call_it():
                nonlocal new_args, new_kwargs, time_of_last_call, scheduled, timer
                time_of_last_call = time()
                fn(*new_args, **new_kwargs)
                scheduled = False
            time_since_last_call = time() - time_of_last_call
            new_args, new_kwargs = args, kwargs
            if not scheduled:
                scheduled = True
                new_wait = max(0, wait - time_since_last_call)
                timer = Timer(new_wait, call_it)
                timer.start()
        return throttled
    return decorator

要了解它与去抖动器的行为有何不同，这里是一个相同的滑块示例，其节流值显示在文本框中。注意它在限制回调速率的同时，交互性有多强。

In [17]:
slider = widgets.IntSlider()
text = widgets.IntText()

@throttle(0.2)
def value_changed(change):
    text.value = change.new
slider.observe(value_changed, 'value')

widgets.VBox([slider, text])